In [11]:
import getData
#import torch
import numpy as np
import pandas as pd
import os
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments



data_path = getData.fetch_dataset()

creepy_pasta_df = pd.read_excel(data_path + "\\" +  os.listdir(data_path)[0])
creepy_pasta_np = np.array(creepy_pasta_df)

print(creepy_pasta_df.head())

                                          story_name  average_rating  \
0  If You’re Armed and at the Glenmont Metro, Ple...            9.25   
1                                   A Shattered Life            9.13   
2      My Father Punished Me When I Talked to Ghosts            9.12   
3                          Distorted Warning Signals            9.12   
4                                     He Who Wanders            9.12   

                                                tags  \
0   drug trials, drugs, experimentation, experime...   
1   disease, entities, Matt Dymerski, monsters, p...   
2   abductions, blindness, child abuse, childhood...   
3                              Ashley Rose Wellman\n   
4                                   Simon Simonian\n   

                                                body estimated_reading_time  \
0  If you’re armed and at the Glenmont metro, ple...             12 minutes   
1    I don’t know when you’re going to read this,...             14 minu

# Data Cleaning

In [12]:
# I'm unsure whether or not the tags and categories will be necessary, for now I'm just cleaning the story text

#print(creepy_pasta_np[:,3][2])

def scrub(array):
    
    vremove_non_space_whitespace = np.vectorize(lambda x: re.sub(r'[\t\r\f\v]+', '', x))
    colorized = vremove_non_space_whitespace(array)
    cleaned = [x.replace("* * * * * *\n", "") for x in colorized]

    return cleaned


cleaned_dataset = scrub(creepy_pasta_np[:,3])
pd.DataFrame(cleaned_dataset)
print(cleaned_dataset[4])


I missed the scorching wind of Andalusia. How it pours sunlight onto your face, toying with eyelashes, flattening dry sand against cheeks and milling around hair. I missed the smell of the valley and that ripening softness of Muscat fluff glistening in the afternoon breeze.
From up here, I can see the house where I grew up. I see white chapels tucked into grape orchards like pawns scattered on a chess board. I can see patches of asphalt on El Jardinito Road hailing from the old town through dappled rocks, then waning behind the horizon with erratic headlights of beat-up trucks cruising along.

One of the pit stops along Ed Jardinito, where truck drivers stop to relieve themselves, marks the starting point to this wavy trail. All covered in blotches of spindly grass stalks and flaxen sand, the trail is barely noticeable at first. Truth is, no one even cares to notice it. Why would truckers taking a blitz-leak care to check on a mucky trail leading to God knows where? But I do. This is h

In [13]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)


# Tokenize Scary Story Dataset
def tokenize(text):
    inputs = tokenizer(text, truncation=True, padding='max_length', max_length=128)
    inputs['labels'] = inputs['input_ids'].copy()
    return inputs

tokenized_data = cleaned_dataset.apply(tokenize)


ImportError: 
AutoModelForCausalLM requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


# Training

In [ ]:
trainingArgs = TrainingArguments(
    output_dir= "Models",
    evaluation_strategy='epoch',
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='Logs'
)
    
'''   
trainer = Trainer(
    model = model,
    args = trainingArgs,
    train_dataset=,
    eval_dataset=
    
)
'''